In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

In [2]:
#Headers para a requisição no ML
headers = {
    "Referer": 'https://lista.mercadolivre.com.br/',
    "Sec-Ch-Ua": "Not_A Brand",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "Windows",
    "User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'    
}

In [12]:
# Item e Quantidade de Itens para a Busca
item = "bicicleta"
num_itens = 20


# URL base para a consulta no Mercado Livre
base_url = f"https://lista.mercadolivre.com.br/{item}"


# Lista para armazenar os itens
itens = []

# Loop while para obtermos o número necessário de itens
while len(itens) < num_itens:
    url = f"{base_url}&_Desde_{len(itens)}_NoIndex_True"   
    
    # Requisição para a URL
    response = requests.get(url, headers=headers)
    
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Procura pelos itens da lista, baseado na classe específica do HTML do Mercado Livre
        item_containers = soup.find_all("li", {"class": "ui-search-layout__item"})
        
        # Loop através dos itens encontrados na página
        for itemMl in item_containers:
            title = itemMl.find("a", {"class": "ui-search-link__title-card"})            
            price_inter = itemMl.find("span", {"class": "andes-money-amount__fraction"}).text.strip() 
            price_cents = itemMl.find("span", {"class": "andes-money-amount__cents"})            
            rating = itemMl.find("span", {"class": "a-icon-alt"}) or "Sem Informação"
            delivery_full = "Disponível" if itemMl.find("use", {"href": "#poly_full"}) else "Indisponível"
            
            if title:
                itemMl_title = title.text.strip()
                price_cents_value = price_cents.text.strip() if price_cents else "0"            
                          
                # Adiciona o item à lista de itens coletados    
                itens.append({
                    "Title": itemMl_title,                        
                    "Price": price_inter + "." + price_cents_value,
                    "Rating": rating,
                    "DeliveryFull": delivery_full,
                })
                # Adiciona o item à lista de itens coletados
                print(itemMl_title)       
        
    else:
        print("Failed to retrieve the page")
        break

# Limita o número de itens coletados ao valor definido por 'num_itens'
itens = itens[:num_itens]

# Converte itens em um DataFrame
df = pd.DataFrame(itens)

# Remove duplicatas no DataFrame, baseando-se nas colunas 'Title' e 'Price'# Remove duplicates based on 'Title' column
df.drop_duplicates(subset=["Title","Price"], inplace=True)



https://lista.mercadolivre.com.br/bicicleta&_Desde_0_NoIndex_True
Bicicleta Aro 26 Caiçara Beach Vintage Bike Retro Feminina
Bicicleta Aro 29 Gts Alumunio Rdx 21v Câmbio Shimano A Disco Cor Branco/preto Tamanho Do Quadro 21
Mountain bike Woltz Steel aro 29 17" 21v freios de disco mecânico câmbios Yamada cor preto/vermelho
Bicicleta Aro 29 Gts Alumunio Rdx 21v Câmbio Shimano A Disco Cor Cinza/preto Tamanho Do Quadro 17
Bicicleta De Mtb St900 Ltd Aro 29 Transmissão Shimano
Bicicleta Mtb Aro 29 Oggi Hacker Hds 2021 Preto/azul/s-lime
Bicicleta Elétrica Toronto Aro 29 250w Atrio - Vm103
Bicicleta Absolute Nero 5 Elite Aro 29 Suspensão Prime A Ar
Bicicleta Aro 29 Gts Alumunio Rdx 21v Câmbio Shimano A Disco Cor Preto/vermelho Tamanho Do Quadro 21
Bicicleta 12 Nathor Fireman Vermelho
Bicicleta Rad7 Mtb Carbon Black Milano 12v Shimano Sus Boost
Bicicleta Pro-x Rustic Pro -aro-26  Freeride-pump Track
Bicicleta Scott Scale 965 Mtb Shimano Slx 2024 Azul Mtb
Bicicleta Rad7 Mtb Carbon Black Milano 1

In [10]:
print(df.count())

Title           19
Price           19
Rating          19
DeliveryFull    19
dtype: int64
